In [8]:
import inspect
from functools import wraps
import time
import datetime

In [9]:
def mag_cache(fn):
    local_cache = {}
    @wraps(fn)
    def wrapper(*args,**kwargs):
        
        def _clear_expire():
            expire_keys = []
            for k ,(_,stamp) in local_cache.items():
                if datetime.datetime.now().timestamp() - stamp > 5: #handling time: delete element exceeding 5 seconds
                    expire_keys.append(k)
            for k in expire_keys:
                local_cache.pop(k)
                
        def _make_key(args,kwargs):  
            sig = inspect.signature(fn)
            params = sig.parameters ## ordered dict
            param_dict = {}

            param_dict.update(zip(params.keys(),args)) #positional parameters
            param_dict.update(kwargs)          #key-word parameters

            for k,v in params.items():   ##default parameters
                if k not in param_dict:
                    param_dict[k] = v.default

            key = tuple(sorted(param_dict.items()))
            return key
        
        _clear_expire()
        key = _make_key(args,kwargs)
        
        if key not in local_cache:
            local_cache[key] = fn(*args,**kwargs),datetime.datetime.now().timestamp()
    
        return local_cache[key]
    return wrapper

@mag_cache
def add(x=4,y=5):
    time.sleep(3)
    return x+y

In [10]:
add()

(9, datetime.datetime(2019, 9, 3, 22, 39, 12, 659501))

In [11]:
add(4,5)

(9, datetime.datetime(2019, 9, 3, 22, 39, 12, 659501))

In [12]:
add(x=4,y=5)

(9, datetime.datetime(2019, 9, 3, 22, 39, 12, 659501))

In [13]:
add(y=5,x=4)

(9, datetime.datetime(2019, 9, 3, 22, 39, 12, 659501))